In [54]:
from pyspark.sql import SparkSession 
from pyspark.sql.functions import *
from datetime import date
from pyspark.sql.types import BooleanType
import pyspark.sql.functions as F

spark = SparkSession.builder.appName("my-spark-app").config("spark.ui.port","5050").getOrCreate()
df = spark.read.format("csv").option("header", "true").load("../../df_matches.csv")
df.printSchema()
df2 = df.withColumnRenamed("X4","match").withColumnRenamed("X6","competition")
df2.printSchema()
df2.select('match', 'competition', 'adversaire', 'score_france', 'score_adversaire', 'penalty_france', 'penalty_adversaire', 'date').show()

def blank_as_zero(x):
    return when(col(x) != "NA", col(x)).otherwise(0)

dfWithEmptyReplaced = df2.withColumn("penalty_france", blank_as_zero("penalty_france")).withColumn("penalty_adversaire", blank_as_zero("penalty_adversaire"))
dfWithEmptyReplaced.show()

today = date.today().strftime("%Y-%m-%d")
dfFiltered = dfWithEmptyReplaced.filter((dfWithEmptyReplaced.date >= '1980-03-01') & (dfWithEmptyReplaced.date <= today))
dfFiltered.show()

def isDomicile(match):
    if match.split('-')[0] == "France":
        res = True
    else:
        res = False
    return res

isDomicile_udf = F.udf(extract_title, BooleanType())

dfFiltered.withColumn('domicile', isDomicile_udf(dfFiltered.match)).show()

dfFiltered.select(avg('score_france').alias('stat1')).write.parquet('stat1.parquet')

root
 |-- X2: string (nullable = true)
 |-- X4: string (nullable = true)
 |-- X5: string (nullable = true)
 |-- X6: string (nullable = true)
 |-- adversaire: string (nullable = true)
 |-- score_france: string (nullable = true)
 |-- score_adversaire: string (nullable = true)
 |-- penalty_france: string (nullable = true)
 |-- penalty_adversaire: string (nullable = true)
 |-- date: string (nullable = true)
 |-- year: string (nullable = true)
 |-- outcome: string (nullable = true)
 |-- no: string (nullable = true)

root
 |-- X2: string (nullable = true)
 |-- match: string (nullable = true)
 |-- X5: string (nullable = true)
 |-- competition: string (nullable = true)
 |-- adversaire: string (nullable = true)
 |-- score_france: string (nullable = true)
 |-- score_adversaire: string (nullable = true)
 |-- penalty_france: string (nullable = true)
 |-- penalty_adversaire: string (nullable = true)
 |-- date: string (nullable = true)
 |-- year: string (nullable = true)
 |-- outcome: string (nullab